# Main notebook

For quick and generic simulations.

## Imports

In [1]:
# %matplotlib widget

import lppydsmc as ld
import plotting

# imports for poisson solver
from fenics import *
import lppydsmc.poisson_solver as ps


# other imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import os
from pathlib import Path

# you can choose the seed here
np.random.seed(1111)

## System choice

Four default systems can be initialized using the following cell.

In [2]:
# ---------------------- System --------------------
system_type = 'thruster'
dz = 0.001
typical_lenght = 0.001 # typical size of the system (minimum distance between two walls for example)
                       # used for computing the mean free path later on (not used in the simulation)
                       # just useful to have an idea of it.

dp = 0.001
# here, x : l, y : w
dict_thruster = {
    'w_in' : 5*dp,
    'l_in' : 3*dp,
    'w_1' : 3*dp,
    'l_1' : dp,
    'l_int' : dp,
    'w_2' : dp,
    'l_2' : 10*dp,
    'w_out' : 5*dp,
    'l_out' : dp,
    'offsets' : np.array([0,0]) 
}

system, idx_out_walls, idx_in_wall = ld.systems.helper.thruster_system(**dict_thruster)
idx_out_walls = [10, 9, 11] # not the input wall
offsets = system.get_offsets()
system_shape = system.system_shape()
a = system.get_dir_vects()
segments = system.get_segments()

In [3]:
# electric field and stuff
charge_density = {
            'value' : '0',  # must be a string too
            'degree' : 0,
            'kwargs' : {}
            }
potential_field, electric_field = ps.helper.thruster(dimensions = dict_thruster, mesh_resolution = 100, \
                                                     potential_electrode_1 = '30', potential_electrode_2 = '300', \
                                                     charge_density = charge_density)


## Particles

In [4]:
# ------------------ Particles Params ----------------- #

# particles density in the real system
density = 1e17 # m-3

part_type = 'I-'
charge, mass, radius = 1*ld.utils.physics.ELECTRON_CHARGE, ld.utils.physics.get_mass_part(53+1, 53, 74), 2e-10

temperature = 24e3 # K - ion speed is much higher (around 2000 m/s)
v_mean = ld.utils.physics.maxwellian_mean_speed(temperature, mass) # for 24k K , ~ 1993 m/s

size_array = 1000 # for example - to start with
container = ld.data_structures.Particle(part_type, charge, mass, radius, size_array)

# particle cross section (useful for particles collision)
cross_section = container.get_params()[3]  

# mean free path and time
mfp = ld.utils.physics.mean_free_path(cross_section, density)
mft = ld.utils.physics.mean_free_time(mfp, v_mean = v_mean) # min(typical_lenght,mfp)

In [5]:
ld.utils.physics.mean_free_time(typical_lenght, v_mean = 10*v_mean) # this is the actual time we need

5.0171189082561205e-08

## Injection params 

In [6]:
inject_particles = True

if(inject_particles):
    in_wall = segments[idx_in_wall]
    in_vect = np.array([a[idx_in_wall,1], -a[idx_in_wall,0]])
    
    # for the injection
    vel_std = ld.utils.physics.gaussian(temperature, mass)


## Simulation parameters

In [7]:
# Simulation params
iterations = int(1e4)
dt = 1e-7 # in sec.

# saving params 
saving_period = 100 # when do we save various data (see the simulation algo)
adding_period = 1 # when to we add to the dataframe that contains the particles position and velocity

# advection function - returns a 2D arrays containing the acceleration for each of the given particle
    # here arr is a N x 5 array. N particles, and for each one : x, y, vx, vy, vz is stored.
    
def f(arr, t, m, q, electric_field):
    der = np.zeros((arr.shape[0], 5))
    fact  = m/q
    for k, part in enumerate(arr):
        try:
            der[k,2:4] =  fact * electric_field(part[:2])
        except Exception as e:
            der[k,2:4] = 0.
    der[:,:2] = arr[:,2:4]
    return der

# args is given to euler_explicit and then given to *f* (the advection function) in addition to arr and dt.
# in our case, it is not needed. However, we could imagine a system with an electric field computed at the setup phase, 
# and we would like to give it as an args.
args = [mass, charge, electric_field]
scheme = ld.utils.schemes.rk4

## Summing-up and plotting

In [8]:
print(f'Initializing a system of type {system_type}, of shape {system_shape}.')
print(f'The number of particles (type {part_type}) in the system is {container.get_current()}.')
print(f'Mean free path : {mfp} m ; Mean free time : {mft} m')
print(f'The simulation lasts {iterations} iterations, with a time step of {dt} s. Simulation duration : {dt*iterations} s')
# Note:  HDF5 uses a different format than csv, and the size on the disk is much different that what is expected. Check out : https://support.hdfgroup.org/HDF5/doc/H5.intro.html
# Here, you can at least multiply by 4 the size (considering we save much more than )
print(f'Disk space usage for saving this simulation (counting ONLY the particles positions and speed) and considering that we save {size_array} particles each time is {iterations//adding_period*size_array*(5*4)/1024**2} MB.') 

Initializing a system of type thruster, of shape [0.016 0.005].
The number of particles (type I-) in the system is 0.
Mean free path : 14.067442439954778 m ; Mean free time : 0.007057803145630173 m
The simulation lasts 10000 iterations, with a time step of 1e-07 s. Simulation duration : 0.001 s
Disk space usage for saving this simulation (counting ONLY the particles positions and speed) and considering that we save 1000 particles each time is 190.73486328125 MB.


# Saving directory and name

In [9]:
# which directory is used to save the data
# and under what name.
dir_path = Path('results/')
name = 'pusher_poisson.h5' 

saver = ld.data.saver.Saver(dir_path, name)

# Simulation

The next cell takes care of the simulation. It algo gives you an idea of the evolution of the number of particles in the system and of its very general state.

At the end, the *saver* which saves the data is closed and you can then analyse your simulation using *analysis.ipynb*.

In [10]:
df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) # bucket for the particles - index of particles is the iteration number
tracking_out = pd.Series(dtype = int)
tracking_collisions = pd.Series(dtype = int)
# ------------------------- INJECTING PARTICLES ------------------------- 
remains = 0
debit = size_array/dt
new, remains = ld.injection.maxwellian(in_wall, in_vect, debit, vel_std, dt, remains) # injecting directly 1000 particles
container.add_multiple(new)

# SIMULATING
print('|{:^10}|{:^10}|{:^10}|'.format(' it ', ' INIT ', 'DEL '))
print('{:-^34}'.format(''))

t = 0.
for it in range(1,iterations+1): # tqdm
    n1 = container.get_current()
 
    # ---------------------------- PHASE : ADVECTING --------------------
        # MOVING PARTICLES
    arr = container.get_array()
        
    ld.advection.advect(arr, f, dt, t, args, scheme) # advect is inplace
    
        # HANDLING BOUNDARIES 
    count = np.full(fill_value = True, shape = arr.shape[0])
    idxes_out = []
    c = 0
    collisions_with_walls = 0
    while(np.count_nonzero(count) > 0):  # np.sum(count, where = count == True) - does not work in python 3.7
        c+=1
        ct, cp = ld.advection.wall_collision.handler_wall_collision_point(arr[count], segments, a) # handler_wall_collision(arr[count], segments, a, radius)
        count, idxes_out_ = ld.advection.wall_collision.make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count) # idxes_out : indexes of the particles (in arr) that got out of the system
        idxes_out.append(idxes_out_)
        
        # the first one that is received is the number of particles colliding with walls.
        if(c == 1):
            collisions_with_walls = np.count_nonzero(count)
    
    idxes_out = np.sort(np.concatenate(idxes_out))
    container.delete_multiple(idxes_out)

    t += dt
    
    # ----------------------------- PLOTTING AND SAVING (OPTIONAL) ----------------------------- 
    if(it%adding_period == 0 or it == iterations-1):
        df = df.append(pd.DataFrame(data=arr, index=[it]*arr.shape[0], columns = ['x','y','vx','vy','vz']))
        tracking_out.loc[it] = len(idxes_out)
        tracking_collisions.loc[it] = collisions_with_walls
    if(it%saving_period == 0 or it == iterations-1): # saving if last iteration too
        saver.save(it = it, append = {
                        'df' : df,
                        'total_deleted' : tracking_out, # evolution of the number of deleted particles per cell (int)
                        'collisions_with_walls' : tracking_collisions, # number of collisions with walls - evolution
                  })
        
        # resetting dataframe to not use too much memory
        df = pd.DataFrame(columns = ['x','y','vx','vy','vz'])
        print('|{:^10}|{:^10}|{:^10}|'.format(it, n1, idxes_out.shape[0]))
    
    if(container.get_current()==0):
        print('All particles got out.')
        break
        
saver.close()

|    it    |   INIT   |   DEL    |
----------------------------------
|   100    |   867    |    1     |
|   200    |   644    |    0     |
|   300    |   499    |    1     |
|   400    |   411    |    2     |
|   500    |   336    |    0     |
|   600    |   284    |    1     |
|   700    |   247    |    0     |
|   800    |   215    |    0     |
|   900    |   199    |    0     |
|   1000   |   179    |    0     |
|   1100   |   164    |    0     |
|   1200   |   151    |    1     |
|   1300   |   137    |    0     |
|   1400   |   121    |    0     |
|   1500   |   112    |    0     |
|   1600   |   105    |    0     |
|   1700   |    99    |    0     |
|   1800   |    93    |    0     |
|   1900   |    88    |    0     |
|   2000   |    84    |    0     |
|   2100   |    81    |    0     |
|   2200   |    78    |    0     |
|   2300   |    73    |    0     |
|   2400   |    66    |    0     |
|   2500   |    63    |    0     |
|   2600   |    61    |    0     |
|   2700   |    58  